# 输出解析器

输出解析器是 LangChain 中的一个重要组件，它主要用于将语言模型的输出转换为结构化数据。

## 主要功能

1. **格式化输出**：将语言模型的文本输出转换为特定的格式，如 JSON、Python 对象等
2. **验证输出**：确保输出符合预期的格式和约束
3. **错误处理**：当输出不符合预期时提供清晰的错误信息

## 内置输出解析器

LangChain 提供了多种内置的输出解析器：

1. `PydanticOutputParser`：将输出解析为 Pydantic 模型
2. `StructuredOutputParser`：将输出解析为结构化数据
3. `CommaSeparatedListOutputParser`：将输出解析为逗号分隔的列表
4. `OutputFixingParser`：自动修复格式错误的输出

In [1]:
import os
import json
import dashscope
from dashscope import Generation
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)

# 与 Qwen 对话
def chat_with_qwen(prompt_value):
    # 从 StringPromptValue 中提取字符串
    if hasattr(prompt_value, "to_string"):
        prompt_str = prompt_value.to_string()
    else:
        prompt_str = str(prompt_value)
    
    response = Generation.call(
        model=config['qwen_model'],
        prompt=prompt_str,
        api_key=config['api_key']
    )
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")

# 创建一个调试回调函数
def print_input(x):
    print(f"提示词：\n{x}")
    return x

# 定义一个 Pydantic 模型
class User(BaseModel):
    name: str = Field(description="用户名")
    age: int = Field(description="年龄")
    height: int = Field(description="身高以厘米为单位")
    weight: int = Field(description="体重以千克为单位")

# 创建一个输出解析器
parser = PydanticOutputParser(pydantic_object=User)

# 创建一个提示模板
prompt = PromptTemplate(
    template="""
    回答用户的询问并以JSON格式返回：
    {format_instructions}
    问题：{question}
""",
input_variables=["question"],
partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 修改链，添加调试步骤
debug_chain = (
    {"question": RunnablePassthrough()} 
    | prompt
    | print_input  # 这里会打印出格式化后的提示词
    | chat_with_qwen 
    | StrOutputParser() 
    | parser
)

# 调用链
result = debug_chain.invoke("阿里巴巴CEO")
print(result)



提示词：
text='\n    回答用户的询问并以JSON格式返回：\n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "用户名", "title": "Name", "type": "string"}, "age": {"description": "年龄", "title": "Age", "type": "integer"}, "height": {"description": "身高以厘米为单位", "title": "Height", "type": "integer"}, "weight": {"description": "体重以千克为单位", "title": "Weight", "type": "integer"}}, "required": ["name", "age", "height", "weight"]}\n```\n    问题：阿里巴巴CEO\n'
name='张勇' age=51 height=176 weight=68


#  结构化提取
## 使用LangChain的相应模式
LangChain的相应模式将做两件事情：
1. 使用真实格式说明，LangChain自动生成提示词
2. 读取LLM的输出并将其转化为适合的Py对象

## 从聊天消息中提取用户想要做的事情

In [9]:
import json
import dashscope
from dashscope import Generation
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate

from langchain.output_parsers import StructuredOutputParser,ResponseSchema

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)

# 与 Qwen 对话
def chat_with_qwen(prompt_value):
    # 从 StringPromptValue 中提取字符串
    if hasattr(prompt_value, "to_string"):
        prompt_str = prompt_value.to_string()
    else:
        prompt_str = str(prompt_value)
    
    response = Generation.call(
        model=config['qwen_model'],
        prompt=prompt_str,
        api_key=config['api_key']
    )
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")

# 定义格式
response_schemas = [
    ResponseSchema(name="菜品",description="菜的名字"),
    ResponseSchema(name="地名",description="该道菜品常见的地名")
]

# 定义结构化输出解析器
StrOutputParser = StructuredOutputParser.from_response_schemas(response_schemas=response_schemas)

format_instructions = StrOutputParser.get_format_instructions()

print(f"结构化输出解析器的提示词:\n {format_instructions}")


prompt = """
    根据用户的输入，提取所有的菜品和地名\n
    {format_instructions}{user_prompt}
"""

promptTemplate = PromptTemplate(
    template = prompt,
    input_variables = ["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

query = promptTemplate.format_prompt(
    user_prompt = """
日记：探索美食与地名的奇妙之旅
日期：2025年4月9日
今天，我踏上了一段令人兴奋的美食之旅，探索了几个独特的地名及其特色美食。
早晨：杭州
清晨，我来到了美丽的杭州。这里的西湖美景令人陶醉，湖边的早茶更是不可错过。我点了一份龙井虾仁，鲜嫩的虾仁搭配着清香的龙井茶，味道鲜美，令人回味无穷。
中午：成都
午餐时，我飞往了成都，享受了正宗的四川火锅。火锅的麻辣味道让我感受到了一种前所未有的刺激，搭配新鲜的蔬菜和肉类，每一口都充满了满足感。成都的街头小吃也让我流连忘返，尤其是兔头，香辣可口，令人欲罢不能。
下午：广州
下午，我来到了广州，享受了一顿地道的早茶。点心如虾饺、烧卖、和蛋挞，每一样都精致可口。我特别喜欢那一口咬下去，皮薄馅嫩的虾饺，仿佛整个早晨的疲惫都被一扫而空。
晚上：上海
晚上，我抵达了上海，漫步在繁华的南京路。这里的美食琳琅满目，我选择了生煎包作为晚餐。外皮酥脆，内馅多汁，搭配醋和姜丝，味道绝妙。
总结
这一天的旅程让我更加热爱中国的美食文化。每一个地方都有其独特的风味和故事，真是一次难忘的体验！期待下一次的美食探索之旅。
""")
print(f"最终的提示词：\n {query}")
print("=========结果===========")
response = chat_with_qwen(query)
print(response)

结构化输出解析器的提示词:
 The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"菜品": string  // 菜的名字
	"地名": string  // 该道菜品常见的地名
}
```
最终的提示词：
 text='\n    根据用户的输入，提取所有的菜品和地名\n\n    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"菜品": string  // 菜的名字\n\t"地名": string  // 该道菜品常见的地名\n}\n```\n日记：探索美食与地名的奇妙之旅\n日期：2025年4月9日\n今天，我踏上了一段令人兴奋的美食之旅，探索了几个独特的地名及其特色美食。\n早晨：杭州\n清晨，我来到了美丽的杭州。这里的西湖美景令人陶醉，湖边的早茶更是不可错过。我点了一份龙井虾仁，鲜嫩的虾仁搭配着清香的龙井茶，味道鲜美，令人回味无穷。\n中午：成都\n午餐时，我飞往了成都，享受了正宗的四川火锅。火锅的麻辣味道让我感受到了一种前所未有的刺激，搭配新鲜的蔬菜和肉类，每一口都充满了满足感。成都的街头小吃也让我流连忘返，尤其是兔头，香辣可口，令人欲罢不能。\n下午：广州\n下午，我来到了广州，享受了一顿地道的早茶。点心如虾饺、烧卖、和蛋挞，每一样都精致可口。我特别喜欢那一口咬下去，皮薄馅嫩的虾饺，仿佛整个早晨的疲惫都被一扫而空。\n晚上：上海\n晚上，我抵达了上海，漫步在繁华的南京路。这里的美食琳琅满目，我选择了生煎包作为晚餐。外皮酥脆，内馅多汁，搭配醋和姜丝，味道绝妙。\n总结\n这一天的旅程让我更加热爱中国的美食文化。每一个地方都有其独特的风味和故事，真是一次难忘的体验！期待下一次的美食探索之旅。\n\n'
=========结果===========
```js

## 自定义大模型解析器
在某些情况下,您可能希望实现自定义解析器以将模型输出构购造为自定义格式
有两种方法可以实现自定义解析器:
- 在LCEL中使用RunnableLambda或RunnableGenerator-我门强烈建议大多数用例使用此方法
- 通过从基类之一继承进行解析--这是困难方法
这两种方法之间的差异大多是表面的,主要在于触发哪些回调(例如,on_chain_start 与 on_parser_start)
## 可运行的Lambda和生成器
推荐的解析方法是使用可运行的lambda和可运行的生成器!

**下面是一段示例代码：用RunnableGenerator封装一个自定义的解析器用于流式输出**

In [14]:
import os
from typing import Iterable
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessageChunk
from langchain_core.runnables import RunnableGenerator

# 定义一个LLM
llm_streaming = ChatOpenAI(
    model= "qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
)

def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        content = chunk.content
        yield content

streaming_output = RunnableGenerator(streaming_parse)

chain = llm_streaming | streaming_output

for chunk in chain.stream("讲一个故事"):
    print(chunk,end="",flush=True)

在一个遥远的山谷里，有一片神秘的森林，人们称它为“星语林”。传说，在这片森林的深处，住着一位会说话的白鹿。白鹿拥有治愈人心的力量，但它只会在月圆之夜现身，并且只会帮助那些真正需要帮助的人。

有一天，一个名叫小岚的女孩误入了星语林。她是一个孤儿，从小被村里的老人抚养长大。虽然生活清贫，但她总是心怀善意，喜欢帮助别人。然而，最近一场突如其来的洪水摧毁了她的家，也让她失去了唯一赖以生存的小木屋。无处可去的小岚只能漫无目的地走进了森林，希望能找到一些食物或者庇护所。

夜幕降临，月亮渐渐升起，银白色的光芒洒满了整个森林。就在小岚感到绝望时，一只通体雪白、双角闪烁着微光的鹿缓缓出现在她面前。它的目光温柔而深邃，仿佛能看透人的内心。

“你为何来到这里？”白鹿用低沉却清晰的声音问道。

小岚惊讶得说不出话来，但还是鼓起勇气回答：“我……我的家被洪水冲毁了，我没有地方可以回去。”

白鹿凝视着她良久，然后轻轻点了点头。“你的善良和坚韧打动了我。我愿意帮你实现一个愿望，但你要记住，这个愿望必须是真正对你和他人有益的。”

小岚愣住了，她从未想过自己还能得到这样的机会。她低头思索片刻，然后抬起头坚定地说：“我希望我的家乡能够恢复生机，让所有失去家园的人都能重新开始新的生活。”

白鹿听后露出了欣慰的笑容，它低下头用蹄子轻触地面，顿时一道柔和的光芒从脚下扩散开来。光芒笼罩了整片森林，随后化作一阵温暖的春风，吹向远方。

第二天清晨，当第一缕阳光照进山谷时，小岚发现四周的一切都焕然一新。河流恢复了清澈，树木重新长出嫩绿的新芽，甚至还有几只小鸟在枝头欢快地歌唱。而更令人惊喜的是，她看到远处的村庄也已重建完毕，村民们正在忙碌地修缮房屋。

从此以后，小岚不仅成为了村子的英雄，还用自己的双手帮助更多需要帮助的人。而关于星语林和白鹿的故事，则一直流传在人们的口耳之间，成为了一个关于希望与善良的美丽传说。